In [1]:
# May need to install some of these imports
import pygame as pg
from pygame.locals import *
import sys
import time
import math
import copy
import random
import datetime
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Game():
# Creates a filled sudoku game board

    def __init__(self, square_length):
        self.square_length = square_length  # Size of each subsquare
        self.side_length = self.square_length*self.square_length   # Dimension of entire board
        self.current_sub_square = 0   # When making game, what subsquare we are currently looking at
        self.board= [0][0]       # Game board
        self.sub_squares = {}    # {sub_square: available numbers} (a sub_square is from left to right, up to down the square that can hold values 1-9)
        self.rows = {}           # {row: available numbers}
        self.columns = {}        # {column: available numbers}
        self.temp_sub_squares = {}     # temporary holder of sub_square values {sub_square: available numbers}
        self.temp_columns = {}    # temporary holder of column values {column: available numbers}
        self.num_found = False    # Whether number has been found for given square
        self.restart = False      # Flag that is true if attempt at making game should be restarted
        self.iterations = 0       # Iterations for trying square that if going over threshold, building game will be restarted
        self.entries = 0          # Nuber of squares successfully filled

    def gameSetup(self):
    # Trying to create a game
    
        self.gameSetupRestart()    # Resetting game

        while(self.entries<self.side_length*self.side_length):   # While game board is not filled
            for row_num in range(self.side_length):   # Going through each row
                col_num = 0
                while(col_num<self.side_length):   # Going through each column
                    self.current_sub_square = math.floor(row_num/self.square_length)*self.square_length + math.floor(col_num/self.square_length)
                    num_found = False

                    while(not num_found):     # Looking for number
                        temp = []    # temporary list of available numbers for a given cell

                        for num in self.rows[row_num]:   # Iterating through available numbers in a given row
                            if num not in temp and num in self.columns[col_num] and num in self.sub_squares[self.current_sub_square]:
                                temp.append(num)

                        if(len(temp)==0):  # Failed attempt where no possible numbers for given cell. Resetting anything changed in this attempt
                            self.sub_squares = self.subSquareReset(self.sub_squares, self.current_sub_square, row_num)
                            self.rows = self.rowReset(self.rows, row_num)
                            self.columns = self.columnReset(self.columns, col_num)
                            self.boardRowReset(row_num)
                            col_num = 0
                            self.current_sub_square = math.floor(row_num/self.square_length)*self.square_length + math.floor(col_num/self.square_length)
                            self.iterations+=1

                            if(self.iterations>self.side_length*self.side_length):   # If numbers of tries is too big, just restart
                                self.restart = True
                                break
                        else:   # Filling square with legal number
                            seed = random.randint(0, len(temp)-1)
                            value = temp[seed]
                            self.board[row_num][col_num] = value
                            self.temp_columns[col_num].append(value)
                            self.temp_sub_squares[self.current_sub_square][math.floor(row_num%self.square_length)].append(value)
                            self.sub_squares[self.current_sub_square].remove(value)
                            self.rows[row_num].remove(value)
                            self.columns[col_num].remove(value)
                            self.entries+=1
                            col_num+=1
                            num_found = True

                    if(self.restart): 
                        break

                for m in range(self.side_length):  # Resetting dict
                    self.temp_columns[m] = []

                if(self.restart):   # Restarting attempt at making game
                    self.gameSetupRestart()
                    break

        if(not self.boardChecker()):  # Double checking that game does not violate rules
            self.gameSetup()
        else:   # If game is good, at 1 to all values to make game 1-indexed instead of 0 (Would be 1-9 for standard game instead of 0-8)
            for i in range(self.side_length):
                for j in range(self.side_length):
                    self.board[i][j]+=1                                      

    def gameSetupRestart(self):
    # Resetting and initializing values for the creation of a completed game board
    
        self.entries = 0
        self.iterations = 0
        self.restart = False
        self.board = [[-1 for i in range(self.side_length)] for i in range(self.side_length)]  # Starting board with all -1's

        # Intializing dicts
        for i in range(self.side_length):
            self.sub_squares[i] = []
            self.rows[i] = []
            self.columns[i] = []
            self.temp_sub_squares[i] = {}
            self.temp_columns[i] = []

        for j in range(self.side_length):
            for k in range(self.side_length):
                self.sub_squares[j].append(k)
                self.rows[j].append(k)
                self.columns[j].append(k)

            for m in range(self.side_length):
                for n in range(self.side_length):
                    self.temp_sub_squares[m][n] = []

    def subSquareReset(self, m, sub_square, row_num):
    # Resetting sub_square dict
    
        num_to_clear = sub_square%self.square_length
        sub_row = row_num%self.square_length

        for i in range(num_to_clear+1):
            temp = copy.deepcopy(self.temp_sub_squares[sub_square-i][sub_row])
            m[sub_square-i].extend(temp)
            self.temp_sub_squares[sub_square-i][sub_row] = []

        return m

    def rowReset(self, m, row_num):
    # Resetting row dict    
        
        m[row_num] = []

        for i in range(self.side_length):
            m[row_num].append(i)
        return m

    def columnReset(self, m, col_num):
    # Resetting column dict    
        for i in range(col_num):
            for num in self.temp_columns[i]:
                m[i].append(num)

            self.temp_columns[i] = []

        return m

    def boardRowReset(self, row_num):
    # Resetting board of game row where failure occurred
    
        for k in range(self.side_length):
            self.board[row_num][k] = -1

    def boardChecker(self):
    # Checking to see if game board is legal    
        
        for i in range(self.side_length):
            for j in range(self.side_length):
                for k in range(j+1, self.side_length):
                    if self.board[i][j]==self.board[i][k] or self.board[j][i]==self.board[k][i]:
                        return False
                    
        self.entries = 0
        row = 0
        col = 0
        modifier = 0
        values = {}

        for p in range(self.side_length):
            values[p] = []

        while(self.entries<self.side_length*self.side_length):
            for i in range(row, row+self.square_length):
                for j in range(col, col+self.square_length):
                    self.entries+=1

                    if len(values[self.board[i][j]])==0:
                        values[self.board[i][j]].append(1)
                    else:
                        return False

            for m in range(self.side_length):
                values[m] = []

            modifier+=1

            if modifier%self.square_length==0:
                col = 0
                row+=self.square_length
            else:
                col+=self.square_length

        return True
    
    def returnBoard(self):
    # Returning game board    
        
        return self.board

    def printBoard(self):
    # Printing game board    
        
        for i in range(self.side_length):
            for j in range(self.side_length):
                print(self.board[i][j], end=" ")
            print(' ')
        print(' ')

In [3]:
class AI():
# AI that solves soduko games. This is being used in the creation of games to make sure they are playable by a human

    def __init__(self, game_board, square_length):
        self.game_board = game_board   # Filled game board
        self.square_length = square_length   # Length of sub_squares (would be 3 in a normal game)
        self.side_length = square_length*square_length    # Length of entire side of board (would be 9 in normal game)
        self.legal_moves = {}    # legal moves for given cell {row: column: list of legal moves}
        self.legal_copy = {}     # nested dict that is a deep copy of legal_moves so we don;t have to reset legal moves when "trying" move
        
        # Initializing legal moves dict
        for i in range(self.side_length):
            self.legal_moves[i] = {}
            
            for j in range(self.side_length):
                if self.game_board[i][j]==0:
                    self.legal_moves[i][j] = []
                    for k in range(1, self.side_length+1):
                        self.legal_moves[i][j].append(k)
                        
    def removeValues(self, board, moves):
    # Removing values from dict that cannot be in given cell
    
        move_made = False
        
        for row in range(self.side_length):
            for col in range(self.side_length):
                if(board[row][col]!=0):
                    current_sub_square = math.floor(row/self.square_length)*self.square_length + math.floor(col/self.square_length)
                    
                    for num in range(self.side_length):
                        if(board[row][num]==0 and board[row][col] in moves[row][num]):
                            moves[row][num].remove(board[row][col])
                            
                        if(board[num][col]==0 and board[row][col] in moves[num][col]):
                            moves[num][col].remove(board[row][col])
                            
                        for s in range(self.side_length):
                            if(board[num][s]==0 
                               and (math.floor(num/self.square_length)*self.square_length + math.floor(s/self.square_length))==current_sub_square 
                               and board[row][col] in moves[num][s]):
                                
                                moves[num][s].remove(board[row][col])
        
        return move_made
    
    def onlyLeft(self, board, moves, verifying):
    # Making move if only 1 possible number can go in a given cell
    
        success = False
        
        for i in range(self.side_length):
            for j in range(self.side_length):
                if(board[i][j]==0 and len(moves[i][j])==1):
                    board[i][j] = moves[i][j][0]
                    moves[i][j] = []
                    
                    if(verifying):
                        return True
                    else:
                        success = True
        
        return success
    
    def checkRow(self, board, moves, verifying):
    # Checking row to see if any moves can be made
    
        num_left = {}
        success = False
        
        for row in range(self.side_length):
            for i in range(1, self.side_length+1):
                num_left[i] = 0
            
            for col in range(self.side_length):
                for num in range(1, self.side_length+1):
                    if(board[row][col]==0 and num in moves[row][col]):
                        val = num_left[num]
                        num_left[num] = val+1
                
            for j in range(1, self.side_length+1):
                if num_left[j]==1:
                    for c in range(self.side_length):
                        if(board[row][c]==0 and j in moves[row][c]):
                            board[row][c] = j
                            moves[row][c] = []
                            
                            if(verifying):
                                return True
                            else:
                                success = True
        
        return success
    
    def checkColumn(self, board, moves, verifying):
    # Checking column to see if any moves can be made    
    
        num_left = {}
        success = False
        
        for col in range(self.side_length):
            for i in range(1, self.side_length+1):
                num_left[i] = 0
            
            for row in range(self.side_length):
                for num in range(1, self.side_length+1):
                    if(board[row][col]==0 and num in moves[row][col]):
                        val = num_left[num]
                        num_left[num] = val+1
                
            for j in range(1, self.side_length):
                if num_left[j]==1:
                    for r in range(self.side_length):
                        if(board[r][col]==0 and j in moves[r][col]):
                            board[r][col] = j
                            moves[r][col] = []
                            
                            if(verifying):
                                return True
                            else:
                                success = True
        
        return success
    
    def checkSubSquare(self, board, moves, verifying):
    # Checking sub square to see if any moves can be made    
    
        num_left = {}
        success = False

        for i in range(0, self.side_length):
            num_left[i] = {}
            
            for j in range(1, self.side_length+1):
                num_left[i][j] = 0
        
        for col in range(self.side_length):
            for row in range(self.side_length):
                current_sub_square = math.floor(row/self.square_length)*self.square_length + math.floor(col/self.square_length)
                
                for num in range(1, self.side_length+1):
                    if(board[row][col]==0 and num in moves[row][col]):
                        val = num_left[current_sub_square][num]
                        num_left[current_sub_square][num] = val+1
        
        for ss in range(self.side_length):
            for j in range(1, self.side_length):
                if num_left[ss][j]==1:
                    for r in range(self.side_length):
                        for c in range(self.side_length):
                            current_sub_square = math.floor(r/self.square_length)*self.square_length + math.floor(c/self.square_length)
                        
                            if(board[r][c]==0 and current_sub_square==ss and j in moves[r][c]):
                                board[r][c] = j
                                moves[r][c] = []
                                
                                if(verifying):
                                    return True
                                else:
                                    success = True
        
        
        return success
    
    def makeMoves(self, board, moves, verifying):
    # Trying to make moves
    
        success = False
        self.removeValues(board, moves)
        
        if self.checkRow(board, moves, verifying):
            self.removeValues(board, moves)
            success = True
            
        if self.checkColumn(board, moves, verifying):
            self.removeValues(board, moves)
            success = True
            
        if self.checkSubSquare(board, moves, verifying):
            self.removeValues(board, moves)
            success = True
            
        if(not self.onlyLeft(board, moves, verifying) and not success):
            return False
        
        return True
    
    def tryMove(self, board, verifying):
    # Iterating through board trying to make moves
    
        success = False
        completed_board = False
        
        for row in range(self.side_length):
            for col in range(self.side_length):
                if board[row][col]==0:
                    remove_nums = []
                    
                    for num in self.legal_moves[row][col]:
                        legal_copy = copy.deepcopy(self.legal_moves)
                        temp_board = copy.deepcopy(self.game_board)
                        temp_board[row][col] = num
                        
                        while(self.makeMoves(board, legal_copy, verifying)):
                            if(self.attemptFailed(temp_board, legal_copy) and verifying):
                                remove_nums.append(num)
                                success = True
                            
                            if(verifying and self.isGameOver(temp_board)):
                                if(not completed_board):
                                    completed_board = True
                                else:
                                    return False
                            
                            if(not verifying and self.isGameOver(temp_board)):
                                self.game_board = copy.deepcopy(temp_board)
                                
                    for num in remove_nums:
                        self.legal_moves[row][col].remove(num)
                        
        return success
    
    def attemptFailed(self, board, moves):
    # Seeing if attempt at move causes the game to have errors
    
        for i in range(self.side_length):
            for j in range(self.side_length):
                if(board[i][j]==0 and len(moves[i][j])==0):
                    return True
                
        return False
    
    def iterateBoard(self, board, verifying):
    # Making moves
    
        while(not self.isGameOver(board) and self.makeMoves(board, self.legal_moves, verifying)):
            pass
            
        if(not self.isGameOver(self.game_board)):
            return False

        return True
    
    def playGame(self, board, verifying):
    # Playing game    
        
        while(not self.isGameOver(board)):
            if(not self.iterateBoard(board, verifying)):
                if(not self.tryMove(board, verifying)):
                    return False
        
        if(self.isGameOver(board)):
            return True
    
        return False
    
    def isGameOver(self, board):
    # Checking to see if game is over    
    
        for row in range(self.side_length):
            for col in range(self.side_length):
                if board[row][col]==0:
                    return False
                
        return True
    
    def returnBoard(self):
    # Returning game board
    
        return self.game_board
    
    def printBoard(self):
    # Printing game board
        for i in range(self.side_length):
            for j in range(self.side_length):
                print(self.game_board[i][j], end=" ")
            print(' ')
        print(' ')

In [4]:
class GameBoard():
# Making game board for human to play

    def __init__(self, filled_board, square_length, difficulty):
        
        self.filled_board = filled_board    # Filled out game board
        self.square_length = square_length  # Square length (3 in normal game)
        self.side_length = square_length*square_length    # Side of board length (9 in normal game)
        self.difficulty = difficulty   # Difficulty of game (either easy, medium, or hard)
        self.game_board = [0][0]       # Game board that will be played by human user
                     
    def makeGame(self):
    # Making the game
    
        threshold = 0   # Liklihood of square to be randomly filled in when creating board
        self.game_board = [[0 for i in range(self.side_length)] for i in range(self.side_length)]   # Initializing game board

        # Liklihood of individual cell to be filled to start game (more likely on easier game modes)
        if(self.difficulty=='e'):
            threshold = (self.square_length-1)/self.square_length - (1/(self.square_length+3))
        elif(self.difficulty=='m'):
            threshold = (self.square_length-1)/self.square_length - (1/(self.square_length+2))
        else:
            threshold = (self.square_length-1)/self.square_length - (1/(self.square_length+1))
        
        # Deciding whether to fill cell or not
        for row in range(self.side_length):
            for col in range(self.side_length):
                
                x = random.uniform(0, 1)
                if(x<threshold):
                    self.game_board[row][col] = self.filled_board[row][col]
                    
        self.ambiguityFinder()  # Removing any obvious ambiguities
  
        test_game_board = [[0 for i in range(self.side_length)] for i in range(self.side_length)]  # making a copy of the game board to be played by AI
        for i in range(self.side_length):
            test_game_board[i] = copy.deepcopy(self.game_board[i])
            
        ai = AI(test_game_board, self.square_length)  # Making AI
        
        if(not ai.playGame(test_game_board, True)):  # If AI cannot complete game, then restart (may be ambiguities)
            self.makeGame()
            
        return self.game_board
    
    def ambiguityFinder(self):
    # Looking to see if game is ambiguous and filling in squares where ambiguity exists (doesn;t find all ambiguities)
    
        current_sub_square = 0
        arr = [-1]*(self.side_length-1)
        
        for m in range(1, self.side_length):
            arr[m-1] = m
            
        for row in range(self.side_length-1):
            for col in range(self.side_length-1):
                current_sub_square = math.floor(row/self.square_length)*self.square_length + math.floor(col/self.square_length)
                
                if self.game_board[row][col]==0:
                    for num in arr:
                        if((row+num)<self.side_length and self.game_board[row+num][col]==0 
                           and current_sub_square==((row+num)/self.square_length)*self.square_length + col/self.square_length):
                            
                            for j in range(col+1, self.side_length):
                                if(self.game_board[row][j]==0 and self.game_board[row+num][j]==0):
                                    
                                    x = random.randint(0, 3)
                                    
                                    if(x==0):
                                        self.game_board[row][col] = self.filled_board[row][col]
                                    elif(x==1):
                                        self.game_board[row+num][col] = self.filled_board[row+num][col]
                                    elif(x==2):
                                        self.game_board[row][j] = self.filled_board[row][j]
                                    else:
                                        self.game_board[row+num][j] = self.filled_board[row+num][j]
                                        
                        if((col+num)<self.side_length and self.game_board[row][col+num]==0 
                           and current_sub_square==((row)/self.square_length)*self.square_length + (col+num)/self.square_length):
                            
                            for j in range(row+1, self.side_length):
                                if(self.game_board[j][col]==0 and self.game_board[j][col+num]==0):
                                    
                                    x = random.randint(0, 3)
                                    
                                    if(x==0):
                                        self.game_board[row][col] = self.filled_board[row][col]
                                    elif(x==1):
                                        self.game_board[row][col+num] = self.filled_board[row][col+num]
                                    elif(x==2):
                                        self.game_board[j][col] = self.filled_board[j][col]
                                    else:
                                        self.game_board[j][col+num] = self.filled_board[j][col+num]                               
                                        
    def returnBoard(self):
    # Returning game board    
    
        return self.game_board
                
    def printBoard(self):
    # Printing game board    
        
        for i in range(self.side_length):
            for j in range(self.side_length):
                print(self.game_board[i][j], end=" ")
            print(' ')
        print(' ')

In [5]:
# Functions for GUI

def openingScreen(width=400, height=600):
# Opening screen that gives user option of which game difficulty to play

    screen = pg.display.set_mode((width, height), RESIZABLE)
    pg.display.update()
    screen.fill(white)
    dim = int(min(width, height))
    board_dim = dim

    if width > height:  # Display easy, medium, hard buttons left to right if screen is wider than it is tall
        pg.draw.rect(screen, (0, 0, 0), (width/10, height/5, 2*width/10, 3*height/5)) 
        pg.draw.rect(screen, (0, 0, 0), (4*width/10, height/5, 2*width/10, 3*height/5)) 
        pg.draw.rect(screen, (0, 0, 0), (7*width/10, height/5, 2*width/10, 3*height/5))
        
        font = pg.font.Font(None, int(dim/12))
        
        easy_text = font.render('easy', 1, (255, 255, 255))
        easy_text_rect = easy_text.get_rect(center=(1*width/5, height/2))
        screen.blit(easy_text, easy_text_rect)
        
        medium_text = font.render('medium', 1, (255, 255, 255)) 
        medium_text_rect = medium_text.get_rect(center=(width/2, height/2))
        screen.blit(medium_text, medium_text_rect)
        
        hard_text = font.render('hard', 1, (255, 255, 255))
        hard_text_rect = hard_text.get_rect(center=(4*width/5, height/2))
        screen.blit(hard_text, hard_text_rect) 
    else:  # Display easy, medium, hard buttons up to down if screen is taller than it is wide
        pg.draw.rect(screen, (0, 0, 0), (width/5, height/10, 3*width/5, 2*height/10)) 
        pg.draw.rect(screen, (0, 0, 0), (width/5, 4*height/10, 3*width/5, 2*height/10)) 
        pg.draw.rect(screen, (0, 0, 0), (width/5, 7*height/10, 3*width/5, 2*height/10))
        
        font = pg.font.Font(None, int(dim/12))
        
        easy_text = font.render('easy', 1, (255, 255, 255))
        easy_text_rect = easy_text.get_rect(center=(width/2, height/5))
        screen.blit(easy_text, easy_text_rect)
        
        medium_text = font.render('medium', 1, (255, 255, 255)) 
        medium_text_rect = medium_text.get_rect(center=(width/2, height/2))
        screen.blit(medium_text, medium_text_rect)
        
        hard_text = font.render('hard', 1, (255, 255, 255))
        hard_text_rect = hard_text.get_rect(center=(width/2, 4*height/5))
        screen.blit(hard_text, hard_text_rect)
            
def closingScreen(width=400, height=600, time='N/A'):
# Closing screen that gives user information on the game they just played and options for new game    
    
    global game_state
    
    screen = pg.display.set_mode((width, height), RESIZABLE)
    pg.display.update()
    screen.fill(white)
    dim = int(min(width, height))
    board_dim = dim
    
    won_font = pg.font.Font(None, int(dim/10))
    
    won_text = won_font.render('You Won!   Time: '+ time, 1, (0, 0, 0))
    won_text_rect = won_text.get_rect(center=(width/2, height/10))
    pg.display.set_caption('Congratulations!')
    screen.blit(won_text, won_text_rect)
    
    new_game_font = pg.font.Font(None, int(dim/10))
    
    new_game_text = new_game_font.render('New Game:', 1, (0, 0, 0))
    new_game_text_rect = new_game_text.get_rect(center=(width/2, 3*height/10))
    screen.blit(new_game_text, new_game_text_rect)
    
    if width > height:
        pg.draw.rect(screen, (0, 0, 0), (3*width/20, 2*height/5, width/5, 3*height/10)) 
        pg.draw.rect(screen, (0, 0, 0), (8*width/20, 2*height/5, width/5, 3*height/10)) 
        pg.draw.rect(screen, (0, 0, 0), (13*width/20, 2*height/5, width/5, 3*height/10))
        
        font = pg.font.Font(None, int(dim/12))
        
        easy_text = font.render('easy', 1, (255, 255, 255))
        easy_text_rect = easy_text.get_rect(center=(width/4, 11*height/20))
        screen.blit(easy_text, easy_text_rect)
        
        medium_text = font.render('medium', 1, (255, 255, 255)) 
        medium_text_rect = medium_text.get_rect(center=(width/2, 11*height/20))
        screen.blit(medium_text, medium_text_rect)
        
        hard_text = font.render('hard', 1, (255, 255, 255))
        hard_text_rect = hard_text.get_rect(center=(3*width/4, 11*height/20))
        screen.blit(hard_text, hard_text_rect) 
    else:
        pg.draw.rect(screen, (0, 0, 0), (3*width/10, 7*height/20, 2*width/5, 3*height/20)) 
        pg.draw.rect(screen, (0, 0, 0), (3*width/10, 11*height/20, 2*width/5, 3*height/20)) 
        pg.draw.rect(screen, (0, 0, 0), (3*width/10, 15*height/20, 2*width/5, 3*height/20))
        
        font = pg.font.Font(None, int(dim/10))
        
        easy_text = font.render('easy', 1, (255, 255, 255))
        easy_text_rect = easy_text.get_rect(center=(width/2, 17*height/40))
        screen.blit(easy_text, easy_text_rect)
        
        medium_text = font.render('medium', 1, (255, 255, 255)) 
        medium_text_rect = medium_text.get_rect(center=(width/2, 5*height/8))
        screen.blit(medium_text, medium_text_rect)
        
        hard_text = font.render('hard', 1, (255, 255, 255))
        hard_text_rect = hard_text.get_rect(center=(width/2, 33*height/40))
        screen.blit(hard_text, hard_text_rect)    
        
def isGameOver():
# Checking to see if game is over

    global game_state, board, filled_board
    
    if(game_state in ['opening', 'closing']):
        return False
    elif(game_state=='game'):
        for i in range(side_length):
            for j in range(side_length):
                if(board[i][j]!=filled_board[i][j]):
                    return False
    return True
    
def userClick():
# Seeing where user clicks mouse and what action to take

    global checked, errors, difficulty, game_state, difficulty, board, game_board, original_board
    global square_length, side_length, start_time, filled_board, board_dim
    
    x, y = pg.mouse.get_pos()
    width = screen.get_width()
    height = screen.get_height()
    dim = int(min(screen.get_width(), screen.get_height()))
    board_dim = dim
    offset = 1 + int(dim/500)
    board_dim = board_dim - 2*offset   # Leave some space on either end for border
    
    
    create_game = False
    
    if(game_state=='opening'):  # If still on opening page, see if user picks difficulty level
        if(width>height):
            if(x>width/10 and x<3*width/10 and y>height/5 and y<4*height/5):
                difficulty = 'e'
                game_state = 'game'
                create_game = True
            elif(x>4*width/10 and x<6*width/10 and y>height/5 and y<4*height/5):
                difficulty = 'm'
                game_state = 'game'
                create_game = True
            elif(x>7*width/10 and x<9*width/10 and y>height/5 and y<4*height/5):
                difficulty = 'h'
                game_state = 'game'
                create_game = True
            else:
                return [True]
        else:
            if(x>width/5 and x<4*width/5 and y>height/10 and y<3*height/10):
                difficulty = 'e'
                game_state = 'game'
                create_game = True
            elif(x>width/5 and x<4*width/5 and y>4*height/10 and y<6*height/10):
                difficulty = 'm'
                game_state = 'game'
                create_game = True
            elif(x>width/5 and x<4*width/5 and y>7*height/10 and y<9*height/10):
                difficulty = 'h'
                game_state = 'game'
                create_game = True
            else:
                return [True]
    elif(game_state=='closing'):  # If still on opening page, see if user picks difficulty level
        if(width>height):
            if(x>3*width/20 and x<7*width/20 and y>2*height/5 and y<7*height/10):
                difficulty = 'e'
                game_state = 'game'
                create_game = True
            elif(x>4*width/10 and x<6*width/10 and y>2*height/5 and y<7*height/10):
                difficulty = 'm'
                game_state = 'game'
                create_game = True
            elif(x>13*width/20 and x<17*width/20 and y>2*height/5 and y<7*height/10):
                difficulty = 'h'
                game_state = 'game'
                create_game = True
            else:
                return [True]
        else:
            if(x>3*width/10 and x<7*width/10 and y>7*height/20 and y<10*height/20):
                difficulty = 'e'
                game_state = 'game'
                create_game = True
            elif(x>3*width/10 and x<7*width/10 and y>11*height/20 and y<14*height/20):
                difficulty = 'm'
                game_state = 'game'
                create_game = True
            elif(x>3*3*width/10 and x<7*width/10 and y>15*height/20 and y<18*height/20):
                difficulty = 'h'
                game_state = 'game'
                create_game = True
            else:
                return [True]
        
    if create_game:  # Start a new game if difficulty is chosen on opening or closing screens
        g = Game(square_length)
        g.gameSetup()
        g.printBoard()
        filled_board = g.returnBoard()
        gb = GameBoard(filled_board, square_length, difficulty)
        game_board = gb.makeGame()
        print('Game Board')
        gb.printBoard()
        board = gb.returnBoard()
        original_board = copy.deepcopy(board)
        start_time = time.time()
        checked = None
        errors = 0
        
        createWindow(width, height)
        return [True]
    else:   # Current game going that may need to be updated with click
        offset = 1 + int(dim/500)
        
        if width > height:
            if width > height*1.5:
                board_dim = dim
            else:
                board_dim = 2*width/3
        elif width <= height:
            if width*1.5 < height:
                board_dim = dim
            else:
                board_dim = 2*height/3
                
        board_dim = board_dim - 2*offset - 1   # Leave some space on either end for border

        col = math.ceil(x/(board_dim/side_length))
        row = math.ceil(y/(board_dim/side_length))
        num_selected = None
        found_num = False
        
        if height >= width:
            if(y>(board_dim+(height-board_dim)/2) and x<=board_dim):
                num_selected = math.ceil(x/(board_dim/side_length))
                found_num = True
        else:
            if(x>(board_dim+(width-board_dim)/2) and y<=board_dim):
                num_selected = math.ceil(y/(board_dim/side_length))
                found_num = True
                
        if(checked==None):
            if(row<=side_length and col<=side_length):
                if(board[row-1][col-1]==0):  # Blank new spot
                    pinkEmpty(row-1, col-1)
                elif(board[row-1][col-1]!=0):  #Numbered new spot
                    yellowNumbers(row-1, col-1)
            if(found_num):
                yellowNumbers(row-1, col-1, num_selected)
            
            
            return [True]
        
        elif(checked==(row-1, col-1)): # If clicking on spot that is highlighted
            if(board[row-1][col-1]!=0):  # If highlihted spots are of number, unhighlight all instances
                whiteNumbers(row-1, col-1)
            elif(board[row-1][col-1]==0):  # If highlighted spot is blank, unhighlight it
                whiteEmpty(row-1, col-1)
                
            checked = None
            return [True]
        
        elif(checked!=(row-1, col-1)):  # If clicking on unhighlighted spot
            if(board[checked[0]][checked[1]]==0): # If previous highlight was on empty space
                whiteEmpty(checked[0], checked[1])
                if(found_num):
                    return(tryNumber(row-1, col-1, num_selected))
                if(row<=side_length and col<=side_length):
                    if(board[row-1][col-1]==0):
                        pinkEmpty(row-1, col-1)
                    else:
                        yellowNumbers(row-1, col-1)
            else:
                whiteNumbers(checked[0], checked[1])
                if(found_num):
                    
                    if(board[checked[0]][checked[1]]!=num_selected):
                        yellowNumbers(row-1, col-1, num_selected)
                    else:
                        checked = None
                if(row<=side_length and col<=side_length):
                    if(board[row-1][col-1]==0):
                        pinkEmpty(row-1, col-1)
                    else:
                        if(board[row-1][col-1]!=board[checked[0]][checked[1]]):
                            yellowNumbers(row-1, col-1)
                        else:
                            checked = None
                 
    return [True]

def userPress():
# Seeing if user clicks on keyboard and what subsequent action to take

    global checked, errors, gb, board, game_board, original_board, board_dim, side_length
    
    if side_length>9:  # Only care about keyboard presses for board size 9 or less
        return [True]
    
    num_selected = None
    
    if pg.key.get_pressed()[K_1]==1 or pg.key.get_pressed()[K_KP1]==1:
        num_selected = 1
    elif pg.key.get_pressed()[K_2]==1 or pg.key.get_pressed()[K_KP2]==1:
        num_selected = 2
    elif pg.key.get_pressed()[K_3]==1 or pg.key.get_pressed()[K_KP3]==1:
        num_selected = 3
    elif pg.key.get_pressed()[K_4]==1 or pg.key.get_pressed()[K_KP4]==1:
        num_selected = 4
    elif pg.key.get_pressed()[K_5]==1 or pg.key.get_pressed()[K_KP5]==1:
        num_selected = 5
    elif pg.key.get_pressed()[K_6]==1 or pg.key.get_pressed()[K_KP6]==1:
        num_selected = 6
    elif pg.key.get_pressed()[K_7]==1 or pg.key.get_pressed()[K_KP7]==1:
        num_selected = 7
    elif pg.key.get_pressed()[K_8]==1 or pg.key.get_pressed()[K_KP8]==1:
        num_selected = 8
    elif pg.key.get_pressed()[K_9]==1 or pg.key.get_pressed()[K_KP9]==1:
        num_selected = 9

    if num_selected!=None:

        if(checked==None):
            yellowNumbers(num_selected=num_selected)
            return [True]

        else:  # If clicking on unhighlighted spot
            if(board[checked[0]][checked[1]]==0): # If previous highlight was on empty space
                whiteEmpty(checked[0], checked[1])

                return(tryNumber(10, 0, num_selected))
            else:
                whiteNumbers(checked[0], checked[1])

                if(board[checked[0]][checked[1]]!=num_selected):
                    yellowNumbers(num_selected=num_selected)
                else:
                    checked = None
                 
    return [True]
                      
def whiteEmpty(i, j):
# Making empty board space white    
    
    global board_dim, side_length, width, height
    
    dim = int(min(width, height))   # dimensions of just the game board
    offset = 1 + int(dim/500)  # For border
    
    # To make sure the colors fit in properly
    x1_adder = 1 + int(dim/500)
    y1_adder = 1 + int(dim/500)
    x2_adder = int(dim/500)
    y2_adder = int(dim/500)

    if i%square_length == 0:
        y1_adder = 2 + int(dim/500)
    if i%square_length == square_length-1:
        y2_adder = 1 + int(dim/500)
    if j%square_length == 0:
        x1_adder = 2 + int(dim/500)
    if j%square_length == square_length-1:
        x2_adder = 1 + int(dim/500)


    posx1 = offset + int(board_dim*(j)/side_length) + x1_adder
    posy1 = offset + int(board_dim*(i)/side_length) + y1_adder
    posx2 = offset + int(board_dim*(j+1)/side_length) - x2_adder
    posy2 = offset + int(board_dim*(i+1)/side_length) - y2_adder

    screen.fill((255, 255, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])
        
def whiteNumbers(row, col):
# Making board space with number white    
    
    global board_dim, side_length, width, height
    
    dim = int(min(width, height))   # dimensions of just the game board
    offset = 1 + int(dim/500)  # For border
    
    for i in range(side_length):
        for j in range(side_length):
            if(board[i][j]==board[row][col]):
                
                # To make sure the colors fit in properly
                x1_adder = 1 + int(dim/500)
                y1_adder = 1 + int(dim/500)
                x2_adder = int(dim/500)
                y2_adder = int(dim/500)

                if i%square_length == 0:
                    y1_adder = 2 + int(dim/500)
                if i%square_length == square_length-1:
                    y2_adder = 1 + int(dim/500)
                if j%square_length == 0:
                    x1_adder = 2 + int(dim/500)
                if j%square_length == square_length-1:
                    x2_adder = 1 + int(dim/500)


                posx1 = offset + int(board_dim*(j)/side_length) + x1_adder
                posy1 = offset + int(board_dim*(i)/side_length) + y1_adder
                posx2 = offset + int(board_dim*(j+1)/side_length) - x2_adder
                posy2 = offset + int(board_dim*(i+1)/side_length) - y2_adder

                screen.fill((255, 255, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

                font_size = int(board_dim/min(30, 10 + 8*(side_length//10)))   # Font size for numbers
                font = pg.font.Font(None, font_size)

                if original_board[i][j]!=0:  # Black if in original board
                    text = font.render(str(board[i][j]), 1, (0, 0, 0))
                else:  # Grey if not in original board
                    text = font.render(str(board[i][j]), 1, (128, 128, 128)) 

                posx = offset + (board_dim/side_length*j+board_dim/side_length*(j+1))/2-(text.get_width()/2)
                posy = offset + (board_dim/side_length*i+board_dim/side_length*(i+1))/2-(text.get_height()/2)
                screen.blit(text, (posx, posy)) 
        
def pinkEmpty(i, j):
# Making empty square that gets clicked pink

    global checked, board_dim, width, height
    
    checked = (i, j)
    dim = int(min(width, height))   # dimensions of just the game board
    offset = 1 + int(dim/500)  # For border
    
    # To make sure the colors fit in properly
    x1_adder = 1 + int(dim/500)
    y1_adder = 1 + int(dim/500)
    x2_adder = int(dim/500)
    y2_adder = int(dim/500)

    if i%square_length == 0:
        y1_adder = 2 + int(dim/500)
    if i%square_length == square_length-1:
        y2_adder = 1 + int(dim/500)
    if j%square_length == 0:
        x1_adder = 2 + int(dim/500)
    if j%square_length == square_length-1:
        x2_adder = 1 + int(dim/500)


    posx1 = offset + int(board_dim*(j)/side_length) + x1_adder
    posy1 = offset + int(board_dim*(i)/side_length) + y1_adder
    posx2 = offset + int(board_dim*(j+1)/side_length) - x2_adder
    posy2 = offset + int(board_dim*(i+1)/side_length) - y2_adder

    screen.fill((255, 204, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1])
    
def yellowNumbers(row=1e7, col=1e7, num_selected=None):
# Making board spaces with given number clicked yellow    
    
    global checked, board_dim, square_length, side_length, width, height
    
    dim = int(min(width, height))   # dimensions of just the game board
    offset = 1 + int(dim/500)  # For border
    
    if(row<side_length and col<side_length):
        checked = (row, col)

    for i in range(side_length):
        for j in range(side_length):
            if((row<side_length and col<side_length and board[i][j]==board[row][col]) or board[i][j]==num_selected):
                if(checked==None or board[checked[0]][checked[1]]!=num_selected):
                    checked = (i, j)
                
                # To make sure the colors fit in properly
                x1_adder = 1 + int(dim/500)
                y1_adder = 1 + int(dim/500)
                x2_adder = int(dim/500)
                y2_adder = int(dim/500)

                if i%square_length == 0:
                    y1_adder = 2 + int(dim/500)
                if i%square_length == square_length-1:
                    y2_adder = 1 + int(dim/500)
                if j%square_length == 0:
                    x1_adder = 2 + int(dim/500)
                if j%square_length == square_length-1:
                    x2_adder = 1 + int(dim/500)

                posx1 = offset + int(board_dim*(j)/side_length) + x1_adder
                posy1 = offset + int(board_dim*(i)/side_length) + y1_adder
                posx2 = offset + int(board_dim*(j+1)/side_length) - x2_adder
                posy2 = offset + int(board_dim*(i+1)/side_length) - y2_adder

                screen.fill((255, 255, 154), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

                font_size = int(board_dim/min(30, 10 + 8*(side_length//10)))   # Font size for numbers 
                font = pg.font.Font(None, font_size)

                if original_board[i][j]!=0:  # Black if in
                    text = font.render(str(board[i][j]), 1, (0, 0, 0))
                else:
                    text = font.render(str(board[i][j]), 1, (128, 128, 128)) 
                
                posx = offset + (board_dim/side_length*(j)+board_dim/side_length*(j+1))/2-(text.get_width()/2)
                posy = offset + (board_dim/side_length*(i)+board_dim/side_length*(i+1))/2-(text.get_height()/2)
                screen.blit(text, (posx, posy))

def tryNumber(i, j, num_selected):
# Making action based of user trying number on a given cell    
    
    global checked, board, filled_board, errors, side_length, width, height
    
    dim = int(min(width, height))   # dimensions of just the game board
    offset = 1 + int(dim/500)  # For border
    
    # To make sure the colors fit in properly
    x1_adder = 1 + int(dim/500)
    y1_adder = 1 + int(dim/500)
    x2_adder = int(dim/500)
    y2_adder = int(dim/500)

    if checked[0]%square_length == 0:
        y1_adder = 2 + int(dim/500)
    if checked[0]%square_length == square_length-1:
        y2_adder = 1 + int(dim/500)
    if checked[1]%square_length == 0:
        x1_adder = 2 + int(dim/500)
    if checked[1]%square_length == square_length-1:
        x2_adder = 1 + int(dim/500)

    posx1 = offset + int(board_dim*(checked[1])/side_length) + x1_adder
    posy1 = offset + int(board_dim*(checked[0])/side_length) + y1_adder
    posx2 = offset + int(board_dim*(checked[1]+1)/side_length) - x2_adder
    posy2 = offset + int(board_dim*(checked[0]+1)/side_length) - y2_adder
    
    font_size = int(board_dim/min(30, 10 + 8*(side_length//10)))   # Font size for numbers
    
    if(checked==None or filled_board[checked[0]][checked[1]]==num_selected):
        if checked!=None:
            #font = pg.font.Font(None, font_size)
            #text = font.render(str(num_selected), 1, (128, 128, 128))
            #posx = (board_dim/side_length*checked[1]+board_dim/side_length*(checked[1]))/2-(text.get_width()/2)
            #posy = (board_dim/side_length*(checked[0]-1)+board_dim/side_length*(checked[0]))/2-(text.get_height()/2)
            #screen.blit(text, (posx, posy))
            board[checked[0]][checked[1]] = num_selected

        for i in range(side_length):
            for j in range(side_length):
                if(board[i][j]==num_selected):
                    if checked==None:
                        checked = (i, j)

                    # To make sure the colors fit in properly
                    x1_adder = 1 + int(dim/500)
                    y1_adder = 1 + int(dim/500)
                    x2_adder = int(dim/500)
                    y2_adder = int(dim/500)

                    if i%square_length == 0:
                        y1_adder = 2 + int(dim/500)
                    if i%square_length == square_length-1:
                        y2_adder = 1 + int(dim/500)
                    if j%square_length == 0:
                        x1_adder = 2 + int(dim/500)
                    if j%square_length == square_length-1:
                        x2_adder = 1 + int(dim/500)


                    posx1 = offset + int(board_dim*(j)/side_length) + x1_adder
                    posy1 = offset + int(board_dim*(i)/side_length) + y1_adder
                    posx2 = offset + int(board_dim*(j+1)/side_length) - x2_adder
                    posy2 = offset + int(board_dim*(i+1)/side_length) - y2_adder

                    screen.fill((255, 255, 154), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

                    font = pg.font.Font(None, font_size)

                    if original_board[i][j]!=0:  # Black if in original board
                        text = font.render(str(board[i][j]), 1, (0, 0, 0))
                    else:   # Gray if not in original board
                        text = font.render(str(board[i][j]), 1, (128, 128, 128)) 

                    posx = offset + int((board_dim/side_length*(j)+board_dim/side_length*(j+1))/2-(text.get_width()/2))
                    posy = offset + int((board_dim/side_length*(i)+board_dim/side_length*(i+1))/2-(text.get_height()/2))
                    screen.blit(text, (posx, posy))    

        return [True]


    else:
        screen.fill((255, 0, 0), rect=[posx1, posy1, posx2-posx1, posy2-posy1])
        errors+=1
        return [False, posx1, posy1, posx2, posy2]
    
def createWindow(width=400, height=600):
# Creating game window

    global checked, game_state, start_time, board_dim, square_length, side_length, board
    
    if(game_state=='opening'):   # If on opening screen
        openingScreen(width, height)
        return
    elif(game_state=='closing'):    # If on closing screen
        seconds = str(int((time.time()-start_time)%60))
        if(len(seconds)==1):
            seconds = '0' + seconds
        closingScreen(width, height, str(math.floor((time.time()-start_time)/60)) + ':' + seconds)
        return
    else:     # If during gameplay
        screen = pg.display.set_mode((width, height), RESIZABLE)
        pg.display.update()
        screen.fill(white)
        dim = int(min(width, height))   # dimensions of just the game board
        offset = 1 + int(dim/500)
        board_dim = dim

        if width > height:
            if width > height*1.5:
                board_dim = dim
            else:
                board_dim = 2*width/3
        elif width <= height:
            if width*1.5 < height:
                board_dim = dim
            else:
                board_dim = 2*height/3
                
        board_dim = board_dim - 2*offset - 1   # Leave some space on either end for border

        # Drawing vertical lines
        for num in range(side_length+1):
            line_thick = 1 + 2*int(dim/500)  # For thinner lines

            if num%square_length==0:
                line_thick = 3 + 2*int(dim/500)  # For thicker lines (borders and subsquare dividers)

                
            # Vertical lines of board
            pg.draw.line(screen, line_color, (offset + int(board_dim*num/float(side_length)), offset), 
                         (offset + int(board_dim*num/float(side_length)), board_dim), line_thick)
            # Horizontal Lines of board
            pg.draw.line(screen, line_color, (offset, offset + int(board_dim*num/float(side_length))), 
                         (board_dim, offset + int(board_dim*num/float(side_length))), line_thick)
            
            if height>=width:  # Game buttons on bottom if height greater than or equal width
                pg.draw.line(screen, line_color, (int(board_dim*num/float(side_length)), int(board_dim+(height-board_dim)/2)), 
                             (int(board_dim*num/float(side_length)), height), 1)
            else:  # Game buttons on side if width greater height
                pg.draw.line(screen, line_color, (int(board_dim+(width-board_dim)/2), int(board_dim*num/float(side_length))), 
                             (width, int(board_dim*num/float(side_length))), 1)

            font_size = int(board_dim/min(30, 10 + 8*(side_length//10)))   # Font size for numbers 
                
            # Putting color background for checked number   
            if(checked!=None):
                if(board[checked[0]][checked[1]]!=0):   # yellow highlighted cells
                    for i in range(side_length):
                        for j in range(side_length):
                            if(board[i][j]==board[checked[0]][checked[1]]):

                                # To make sure the colors fit in properly
                                x1_adder = 1 + int(dim/500)
                                y1_adder = 1 + int(dim/500)
                                x2_adder = int(dim/500)
                                y2_adder = int(dim/500)

                                if i%square_length == 0:
                                    y1_adder = 2 + int(dim/500)
                                if i%square_length == square_length-1:
                                    y2_adder = 1 + int(dim/500)
                                if j%square_length == 0:
                                    x1_adder = 2 + int(dim/500)
                                if j%square_length == square_length-1:
                                    x2_adder = 1 + int(dim/500)


                                posx1 = offset + int(board_dim*(j)/side_length) + x1_adder
                                posy1 = offset + int(board_dim*(i)/side_length) + y1_adder
                                posx2 = offset + int(board_dim*(j+1)/side_length) - x2_adder
                                posy2 = offset + int(board_dim*(i+1)/side_length) - y2_adder

                                screen.fill((255, 255, 154), rect=[posx1, posy1, posx2-posx1, posy2-posy1])

                                font = pg.font.Font(None, font_size)

                                if original_board[i][j]!=0:  # Black if in 
                                    text = font.render(str(board[checked[0]][checked[1]]), 1, (0, 0, 0))
                                else:
                                    text = font.render(str(board[checked[0]][checked[1]]), 1, (128, 128, 128)) 

                                posx = offset + int((board_dim/side_length*(j)+board_dim/side_length*(j+1))/2-(text.get_width()/2))
                                posy = offset + int((board_dim/side_length*(i)+board_dim/side_length*(i+1))/2-(text.get_height()/2))
                                screen.blit(text, (posx, posy))
                else:   # pink highlighted cells
                    
                    # To make sure the colors fit in properly
                    x1_adder = 1 + int(dim/500)
                    y1_adder = 1 + int(dim/500)
                    x2_adder = int(dim/500)
                    y2_adder = int(dim/500)

                    if checked[0]%square_length == 0:
                        y1_adder = 2 + int(dim/500)
                    if checked[0]%square_length == square_length-1:
                        y2_adder = 1 + int(dim/500)
                    if checked[1]%square_length == 0:
                        x1_adder = 2 + int(dim/500)
                    if checked[1]%square_length == square_length-1:
                        x2_adder = 1 + int(dim/500)


                    posx1 = offset + int(board_dim*(checked[1])/side_length) + x1_adder
                    posy1 = offset + int(board_dim*(checked[0])/side_length) + y1_adder
                    posx2 = offset + int(board_dim*(checked[1]+1)/side_length) - x2_adder
                    posy2 = offset + int(board_dim*(checked[0]+1)/side_length) - y2_adder

                    font = pg.font.Font(None, font_size)
                    screen.fill((255, 204, 255), rect=[posx1, posy1, posx2-posx1, posy2-posy1]) 

            # Putting numbers in        
            for n in range(1, side_length+1):
                if board[num-1][n-1]!=0:
                    font = pg.font.Font(None, font_size)
                    text = font.render(str(board[num-1][n-1]), 1, (0, 0, 0))
                    posx = offset + int((board_dim/side_length*(n-1)+board_dim/side_length*(n))/2 - (text.get_width()/2))
                    posy = offset + int((board_dim/side_length*(num-1)+board_dim/side_length*(num))/2 - (text.get_height()/2))
                    screen.blit(text, (posx, posy))

        for num in range(side_length+1):
            if height>=width:   # Numbers at bottom if height greater than or equal to width
                pg.draw.line(screen, line_color, (0, int(board_dim+(height-board_dim)/2)), (board_dim, int(board_dim+(height-board_dim)/2)), 1)
                pg.draw.line(screen, line_color, (0, height), (board_dim, height), 1)

                for num in range(1, side_length+1):
                    font = pg.font.Font(None, font_size)
                    text = font.render(str(num), 1, (0, 0, 0))
                    posx = offset + int((board_dim/side_length*(num-1)+board_dim/side_length*(num))/2 - (text.get_width()/2))
                    posy = offset + int((board_dim+(height-board_dim)/2 + height)/2 - (text.get_height()/2))
                    screen.blit(text, (posx, posy))
            else:   # Numbers on side if width greater than height
                pg.draw.line(screen, line_color, (int(board_dim+(width-board_dim)/2), 0), (int(board_dim+(width-board_dim)/2), board_dim), 1)
                pg.draw.line(screen, line_color, (width, 0), (width, board_dim), 1)

                for num in range(1, side_length+1):
                    font = pg.font.Font(None, font_size)
                    text = font.render(str(num), 1, (0, 0, 0))
                    posx = offset + int((board_dim+(width-board_dim)/2 + width)/2 - (text.get_width()/2))
                    posy = offset + int((board_dim/side_length*(num-1)+board_dim/side_length*(num))/2 - (text.get_height()/2))
                    screen.blit(text, (posx, posy))
        
def resetGame():
# Resetting game    
    
    global board

    g = Game(square_length)
    g.gameSetup()
    g.printBoard()
    filled_board = g.returnBoard()

    gb = GameBoard(filled_board, square_length, 'h')
    game_board = gb.makeGame()
    print('Game Board')
    gb.printBoard()
    board = gb.returnBoard()

In [7]:
#initialize global variables
checked = None
errors = 0    # Number of errors by user
width = 400   # Board width in pixels
height = 600  # Board height in pixels
board_dim = int(min(width, height))   # Dimension of just the game board in pixels
white = (255, 255, 255)   # color of empty window
line_color = (10,10,10)   # line color
square_length = 3   # Size of each square in actual game (3 in normal game)
side_length = square_length*square_length   # Dimension of entire board (9 in normal game)
filled_board = None   # Filled out game board
gb = None
game_board = None
board = None
original_board = None     # Original game board


pg.init()
fps = 30
start_time = None
CLOCK = pg.time.Clock()
screen = pg.display.set_mode((width, height), pg.RESIZABLE)
pg.display.set_caption('Sudoku')

difficulty = 'e'
game_state = 'opening'

openingScreen()

CLOCK = pg.time.Clock()
wrong_num_data = [None]


# run the game loop forever
while(True):
    if(start_time!=None):
        seconds = str(int((time.time()-start_time)%60))
        if(len(seconds)==1):
            seconds = '0' + seconds
        
        if(game_state not in ['opening', 'closing']):
            pg.display.set_caption(str(math.floor((time.time()-start_time)/60)) + ':' + seconds + '       errors: ' + str(errors))
        if(wrong_num_data[0]!=None and (pg.time.get_ticks()-wrong_num_data[0]>500)):  # Filling wrong board space with white
            pg.time.delay(500)
            screen.fill((255, 204, 255), rect=[wrong_num_data[1], wrong_num_data[2], wrong_num_data[3], wrong_num_data[4]])
            wrong_num_data = [None]
        
    for event in pg.event.get():
        
        if event.type == pg.QUIT:
            pg.quit()
            sys.exit()
        elif event.type == VIDEORESIZE:
            width = event.w
            height = event.h
            
            if width < 300:
                width = 300
            if height < 200:
                height = 200
            
            createWindow(width, height)
        elif event.type == pg.MOUSEBUTTONDOWN:
            u = userClick()
            if(not u[0]): # If user picked wrong number, set data in list to time of wrong number and coordinates
                wrong_num_data = [copy.deepcopy(pg.time.get_ticks()), u[1], u[2], u[3]-u[1], u[4]-u[2]]
        elif event.type == pg.KEYDOWN:
            u = userPress()
            if(not u[0]): # If user picked wrong number, set data in list to time of wrong number and coordinates
                wrong_num_data = [copy.deepcopy(pg.time.get_ticks()), u[1], u[2], u[3]-u[1], u[4]-u[2]]

        if(isGameOver()):
            game_state = 'closing'
            createWindow(width, height)
            
    pg.display.update()
    CLOCK.tick(fps)

11 2 14 5 8 4 3 12 15 13 9 10 7 1 6 16  
15 8 13 6 7 1 5 10 12 16 2 14 3 11 4 9  
7 10 16 3 9 11 14 2 1 5 6 4 12 8 15 13  
4 12 1 9 15 16 13 6 3 7 8 11 2 14 10 5  
10 16 8 1 4 15 11 14 9 12 13 6 5 7 3 2  
5 3 12 4 16 9 1 7 2 10 11 15 14 6 13 8  
6 14 15 2 13 3 12 8 7 4 16 5 11 9 1 10  
13 9 7 11 2 10 6 5 8 14 3 1 15 16 12 4  
14 6 9 7 11 12 16 13 10 15 4 3 8 5 2 1  
12 15 5 10 6 2 8 1 14 9 7 16 4 13 11 3  
8 4 2 13 3 5 9 15 6 11 1 12 16 10 7 14  
1 11 3 16 14 7 10 4 13 8 5 2 6 15 9 12  
3 1 4 12 10 13 7 11 5 6 14 8 9 2 16 15  
9 7 11 15 5 14 2 16 4 1 12 13 10 3 8 6  
16 13 6 14 12 8 15 3 11 2 10 9 1 4 5 7  
2 5 10 8 1 6 4 9 16 3 15 7 13 12 14 11  
 
Game Board
11 0 14 5 8 4 0 0 15 0 0 10 7 0 6 16  
15 8 13 0 0 1 0 10 0 0 2 14 0 11 4 9  
7 10 16 3 9 0 14 0 1 0 6 0 12 8 0 13  
0 12 0 9 15 16 13 6 0 7 8 11 2 14 10 0  
0 16 8 1 0 0 0 14 9 0 0 6 5 7 0 2  
5 0 12 4 16 0 0 0 2 0 0 15 14 0 13 8  
0 14 15 2 13 0 0 8 7 4 16 5 0 0 0 0  
13 9 0 11 2 10 6 5 8 14 3 1 0 16 0 4  
14 6 9 7 0 0 16 0 10 

KeyboardInterrupt: 